# Imports

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.applications.mobilenet_v2 import preprocess_input
from keras.utils import img_to_array
from keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [3]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    # grab the dimensions of the frame and construct blob

    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)

    # initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
        
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    return (locs, preds)

In [4]:
# load our serialized face detector model from disk
prototxtPath = r"face_detector\\deploy.prototxt"
weightsPath = r"face_detector\\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [5]:
# load the face mask detector model from disk
maskNet = load_model("mask_detector.model")

In [6]:
vs = VideoStream(src=0).start()

In [7]:
while True:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 400 pixels
	frame = vs.read()
	frame = imutils.resize(frame, width=400)

	# detect faces in the frame and determine if they are wearing a
	# face mask or not
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	# loop over the detected face locations and their corresponding
	# locations
	for (box, pred) in zip(locs, preds):
		# unpack the bounding box and predictions
		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred

		# determine the class label and color we'll use to draw
		# the bounding box and text
		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		# include the probability in the label
		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		# display the label and bounding box rectangle on the output
		# frame
		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	# show the output frame
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

(1, 1, 200, 7)
1/1 [==============================] - 4s 4s/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 33ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 34ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 33ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 32ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 33ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 29ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 33ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 29ms/st

In [8]:
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()